In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

lst = []
# 크롬 설정
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 감지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service(executable_path="chromedriver.exe")  # 경로 확인

driver = webdriver.Chrome(service=service, options=options)

# 자동화 감지 우회용 자바스크립트
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})

# 1. 네이버 검색
url = "https://search.naver.com/search.naver?query=법인+차량"
driver.get(url)

try:
    # 2. FAQ 더보기 버튼이 로딩될 때까지 대기
    faq_more = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "#main_pack > section.sc_new.sp_intent_faq._au_sp_intent_faq._prs_exB_soT > div > div.mod_more_wrap > a"
        ))
    )

    # 3. 마우스 오버 후 클릭 (사람처럼)
    actions = ActionChains(driver)
    actions.move_to_element(faq_more).pause(1).click().perform()

    # 4. FAQ 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "strong"))
    )
    
    time.sleep(1.5)  # 자연스러운 대기

    # 5. 질문 제목 출력 및 "?" 포함 항목만 따로 저장
    titles = driver.find_elements(By.CSS_SELECTOR, "strong")
    question_titles = []

    for title in titles:
        text = title.text
        # print(text)
        if "?" in text:
            question_titles.append(text)

    print("'?'가 포함된 질문 리스트:")
    for q in question_titles:
        lst.append(q)
        print("-", q)

except Exception as e:
    print("에러 발생:", e)

driver.quit()

'?'가 포함된 질문 리스트:
- 법인으로 차량 이전등록시 필요한 서류와 절차는 어떻게 되나요?
- 법인이 차량을 매수할 때 필요한 서류는 무엇인가요?
- 법인차량의 사용범위는 어떻게 되나요?
- 법인이 차량을 구입할 때 어떤 방법이 유리한가요?
- 법인차량을 매각할때 절차는 어떻게 되나요?
- 법인차량의 명의이전은 어디서 할수있나요?
- 법인차량구입시 어떤 이점이 있나요?
- 법인차량 구입시 증빙자료는 뭐가 필요한가요?
- 법인이 차량을 현금으로 구매할 때, 고려해야 할 사항은 무엇인가요?
- 법인이 차량을 등록할 때 내는 세금은 무엇인가요?
- 법인 차량 담보대출의 조건은 어떻게 되나요?
- 법인차량을 구입하면 어떤 혜택이 있나요?
- 법인차량 명의 변경시 필요한 절차는 무엇인가요?
- 법인차량은 어떤 제한 조건이 없나요?
- 법인차량을 운용하면 어떤 절세 효과가 있나요?
- 법인차량은 어떤 자동차보험을 가입해야 세금혜택을 받을수 있나요?
- 법인차량의 취득가액은 어떻게 산정하나요?
- 법인차량 구매방법은 어떻게 되나요?
- 법인차량을 구매할때 리스와 현금구매중 어떤게 유리 한가요?
- 법인이 차량 리스 등을 이용하는 이유는 무엇인가요?


In [5]:
import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({
    "질문": lst,
    "질문에 대한 대답": [""] * len(lst)  # 빈 문자열로 초기화
})

df.to_csv('법인 차량 FAQ 질문.csv', index=False, encoding='utf-8-sig')
print("CSV 파일 저장 완료")


CSV 파일 저장 완료
